## Calculating Distance to Default

**Oliver Seager <br>
o.j.seager@lse.ac.uk <br>
Python 3.9.7**

**Created:** 03/12/2021 <br>
**Last Modified:** 08/04/2021

We use the Black-Scholes-Merton model (Merton, 1974) to infer distance to default using a methodology from Vassalou and Xing (2004), Bohn and Crosbie (2003), and Bharath and Shumway (2008). 

With equity $E$, debt $D$, risk-free interest rate $r$ and time horizon $T$ observable, we use an iterative procedure to back out value $V$ and daily volatility $\sigma_V$ from the below relation of equity and debt to value...

$$
\begin{aligned}
\delta_1 &:= \frac{\log\left(\frac{V}{D}\right) + \left(r + \frac{1}{2}\sigma_V^2 \right)T}{\sigma_V \sqrt{T}}, \\
~\\
\delta_2 &:= \frac{\log\left(\frac{V}{D}\right) + \left(r - \frac{1}{2}\sigma_V^2 \right)T}{\sigma_V \sqrt{T}}, \\
~\\
E &= V\Phi(\delta_1) - e^{-r}D\Phi(\delta_2).
\end{aligned}
$$

...where $\Phi$ gives the cumulative distribution function of the standard normal distribution.

We take $T$ to be 253 - roughly the number of trading days each year in the NYSE...

$$
T = 253.
$$

We take $r$ to be the market daily yield of a 1-year U.S. treasury bill, inferred from annual yields obtained from The Board of Governors of the Federal Reserve via [FRED](https://fred.stlouisfed.org/series/DGS1). Since our annual time horizon consists of 253 trading days, we use...

$$
1 + r = (1 + r_{fed})^{\frac{1}{253}}.
$$

...where $r_{fed}$ gives the Federal Reserve's reported market annual yield.

Suppose we have $N + 1$ observations for the given cusip-datadate with $t=0,\ldots,N$, such that we have $N$ observations for the first differences in variables. We then define daily drift $\mu$ to be given by...

$$
\begin{aligned}
\mu &= \frac{1}{N}\sum_{t=1}^N \Delta \log V_t \\
&= \frac{1}{N}(\log V_N - \log V_0).
\end{aligned}
$$

Daily volatility $\sigma_V$ is then given by...

$$
\sigma_V = \left(\frac{1}{N-1}\sum_{t=1}^N(\Delta \log V_t - \mu)^2\right)^{\frac{1}{2}}.
$$

In our equity-debt-value relation, we have a nonlinear equation in 2 unknowns - $V$ and $\sigma_V$. We proceed iteratively. 

We start by calculating $\mu_E$ and $\sigma_E$ per the above (with equity $E_t$ in place of value $V_t$), and use $\sigma_E$ as our initial value for $\sigma_V$. We then use this to calculate new $V_t$ for the entire cusip-datadate time series. We use this to generate a new $\mu$ and a new $\sigma_V$. We repeat until convergence on $\sigma_V$, with a convergence threshold of 0.0001 following Vassalou and Xing (2004).

Finally, we back out the last value of $V$ in the time series (the one applicable to the Compustat datadate) and use this to calculate distance to default $DD$ and probability of default in the next year $\pi_D$ (naturally, these statistics hold only under Gaussianity - Gaussianity of firm value movements is an assumption that the real world violates repeatedly), given by...

$$
\begin{aligned}
DD &= \frac{\log\left(\frac{V}{D}\right) + \left(\mu - \frac{1}{2}\sigma_V^2 \right)T}{\sigma_V \sqrt{T}}, \\
~\\
\pi_D(DD) &= \Phi(-DD).
\end{aligned}
$$

We use probability of default as well as distance to default as many listed firms report zero debt. This gives an infinite distance to default, which cannot be used in standard linear regression, whilst $\pi_D(\infty)=0$ can.

### Preamble

In [ ]:
from numpy import *
from scipy.optimize import *
from scipy.stats import *
import os
import pandas as pd
from time import time

### Define Function to Back Out $V$

Since we use `scipy.optimize` to solve the nonlinear equity-debt-value relation for $V$ given our present estimate of $\sigma_V$ (`new_sigma_v`), as well as observed $E$ (`equity`), $D$ (`debt`), $r$ (`r`) and $T$ (`t`), we define a function to argue into `fsolve`.

Note that by our equity-debt-value relation, we have...

$$
\lim_{D \to 0^+} \delta_1 = \lim_{D \to 0^+} \delta_2 = \infty
$$

Thus, we have...
$$
V|_{D = 0} = E.
$$

In [ ]:
def find_v(v):
        
    if debt == 0:
                
        # If debt = 0 then the first cdf value is 1 (as log(v/debt) goes to infinity). And the second is irrelevant.
            
        F = equity - v # Will solve as v = equity
            
    else:
            # This is the equity-debt-value  relation
            
        F = equity - (v*norm.cdf((log(v) - log(debt) + (r + 0.5*(new_sigma_v**2))*t)/(new_sigma_v*(t**0.5))) + debt*exp(-1*r)*norm.cdf((log(v) - log(debt) + (r - 0.5*(new_sigma_v**2))*t)/(new_sigma_v*(t**0.5))))
    
    return F

### Get List of Coefficients for Initial Guess on Equity and Debt

A cursory sample of the data suggest that we find a convergent value for $\sigma_V$ where...
$$
V = E + \gamma D,~~~\gamma \in [0,1].
$$

Hence we loop through several initial guesses of $V$ for the algorithmic procedure of `fsolve` below, which generates values of $V$ for a given $\sigma_V^2$. We do this following the below pattern for $\gamma$...
$$
\frac{1}{2},0,1,\underbrace{\frac{1}{4},\frac{3}{4}}_{n_{odd}\cdot2^{-2}},\underbrace{\frac{1}{8},\frac{3}{8},\frac{5}{8},\frac{7}{8}}_{n_{odd}\cdot 2^{-3}},\ldots,\underbrace{\frac{1}{1024},\ldots,\frac{1021}{1024}}_{n_{odd}\cdot2^{-10}}.
$$

Here we generate this list of potential gammas.

In [ ]:
gammas = [0.5,0,1] # Initial list

for i in range(2,10):
    
    reciprocal = 2**(-1*i)
    
    for j in range(1, (2**i), 2):
        
        gammas.append(j*reciprocal)

### Calculate Distance to Default

This horrendous loop is intended to be run with cloud computing - I split the entire distance to default source data into 20 tranches in *001c_data_processing.do* to this end. 

In [ ]:
#########################
# LOOP THROUGH TRANCHES #
#########################

tranches = [1,5,9,13,17] # The specific partitions of the data we want to act on

for tranche_nr in tranches:

    ###################################################
    # CHECK FOR EXISTENCE OF PARTIALLY COMPLETED DATA #
    ###################################################

    if ("001d_dd_complete_tranche" + str(tranche_nr) + ".dta") not in os.listdir():

        # Export an empty dataframe if a "complete" dataframe

        pd.DataFrame(columns = ["cusip","datadate","dd", "p_d"]).to_stata("001d_dd_complete_tranche" + str(tranche_nr) + ".dta")


    ########################################
    # INITIATE DATAFRAMES (REMAINING DATA) #
    ########################################

    df = pd.read_stata("001_c_dd_tranche" + str(tranche_nr) + ".dta") # All of the source data.

    all_unique_cds = df[["cusip","datadate"]].drop_duplicates() # Unique cusip-datadate pairs from the source data

    df_dd = pd.read_stata("001d_dd_complete_tranche" + str(tranche_nr) + ".dta").drop("index", axis = 1) 
                                                                                 # The dataframe to export, containing cusip,
                                                                                # datadate, distance to default, and 
                                                                               # probability of default in  next year. This is 
                                                                              # initated as the already completed portion of the
                                                                             #  dataframe

    complete_unique_cds = df_dd[["cusip","datadate"]].drop_duplicates() # Already completed cusip-datadate pairs

    unique_cds = pd.concat([all_unique_cds, complete_unique_cds]).drop_duplicates(keep = False) # A dataframe of *incomplete*
                                                                                               # cusip-datadate pairs

    #######################
    # FURTHER INITIATIONS #
    #######################

    t = 253 # Always holds - the assumed number of trading days in a year.

    counter = len(all_unique_cds.index) - len(unique_cds.index) # Just a counter for counting

    time1 = time()


    #########################################
    # LOOP - CALCULATE DISTANCES TO DEFAULT #
    #########################################

    for cd in unique_cds.values: # Loops through unique cusip-datadate pairs

        # GET INITIAL ESTIMATE OF LOG_V #

        df_cd = df[(df["cusip"] == cd[0]) & (df["datadate"] == cd[1])] # This dataframe contains the time series for the 
                                                                      # specific cusip-datadate

        log_vs = log(df_cd["equity"]).values # Initial estimates of the V_t


        ## CALCULATE MU ##

        nr_diff_obs = len(log_vs[~isnan(log_vs)]) - 1 # Null observations only appear in the last row, so this is valid.

        log_V_0 = log_vs[~isnan(log_vs)][0] # First observation of log_v (not delta log_v)

        log_V_N = log_vs[~isnan(log_vs)][-1] # Last observation of log_v

        mu = (log_V_N - log_V_0)/nr_diff_obs # Calculate daily drift


        ## CALCULATE INITIAL ESTIMATE OF SIGMA_V ##

        # Since equity is only calculable on trading days, debt figures are released every 3 months and *then*
        # interpolated, and dgs1 is present for every day, I calculate delta log v as 'between consecutive trading days'.

        delta_log_vs = (log_vs - roll(log_vs, 1))[~isnan(log_vs - roll(log_vs, 1))]

        datadate_equity = df_cd["equity"][~isnan(df_cd["equity"])].values[-1]

        datadate_debt = df_cd["debt"].values[-1]

        new_sigma_v = std(delta_log_vs - mu)*(datadate_equity/(datadate_equity + datadate_debt)) # Initial estimate of sigma_v,
                                                                                                # following Bharath and Shumway
                                                                                               # (2008)

        old_sigma_v = 2*new_sigma_v # Prevents convergence in the first iteration
        

        ## ITERATION ##

        iteration_count = 1

        sigma_list = []

        sigma_loop_switch = 0 # Used if sigmas do not converge and estimation gets stuck in a loop

        non_convergence_switch = 0 # Used if unpatterned non-convergence occurs

        estimation_failure_switch = 0 # Used if estimation of v fails for any value of v in the sample
        
        zero_sigma_switch = 0 # Used if neither debt nor equity 

        while True: # Loop broken upon convergence below
            
            # CHECK FOR ZERO VARIANCE #
            
            if new_sigma_v == 0: # This will only ever execute if equity remains the same throughout the period
                
                if len(df_cd["debt"].unique()) > 1: # If debt fluctuates but equity doesn't we can still estimate variance
                                                    # of v, just not with the variance of equity as our initial estimate
                    
                    new_sigma_v = 0.005*(mean(df_cd["debt"])*0.5 + df_cd["equity"].unique()[0])
                    
                    old_sigma_v = 2*new_sigma_v
                
                else: # If debt doesn't fluctuate and neither does equity, then distance to default is infinite
                    
                    zero_sigma_switch = 1
                    
                    break
            
            # CALCULATE NEW Vs #

            v_set = []

            for row in df_cd.values:

                debt, equity, r = row[3:6]

                if pd.isnull(equity):

                    v_set.append(nan)

                else:

                    v = None # Initiate value of v

                    for coef in gammas:

                        if fsolve(find_v, equity + coef*debt, full_output = True)[2] == 1: # Executes if solved

                            v = fsolve(find_v, equity + coef*debt)[0]

                            break

                    if v == None:

                        try: # Last resort for starting guess employed. Probably not sustainable.

                            if fsolve(find_v, prev_v, full_output = True)[2] == 1: # Executes if solved

                                print("Last resort employed")

                                v = fsolve(find_v, prev_v)[0]

                            else:

                                print("Estimation failure")

                                v = None

                                estimation_failure_switch = 1

                        except:

                            print("Estimation failure")

                            v = None

                            estimation_failure_switch = 1

                    v_set.append(v)

                    prev_v = v # Used as last resort for starting guess

            df_cd["v"] = array(v_set)

            # CHECK FOR ESTIMATION FAILURE # This happens in perhaps 0.1% of cases

            if estimation_failure_switch == 1:

                break

            # CHECK FOR CONVERGENCE # This is essentially on our *previous* calculation of sigma_v

            if abs(new_sigma_v/old_sigma_v - 1) < 0.0001:

                break

            # CHECK FOR SIGMA ESTIMATE LOOP #

            # Since the convergence threshold is 10^-4, I use 4*2=8 significant figures when checking for loops

            eightSF_new_sigma_v = round(new_sigma_v, 8 - (int(math.floor(math.log10(abs(new_sigma_v)))) + 1))

            if eightSF_new_sigma_v in sigma_list:

                print("Sigma estimate loop")

                sigma_loop_switch = 1

                loop_sigmas = sigma_list[sigma_list.index(eightSF_new_sigma_v):]

                new_sigma_v = mean(loop_sigmas)

                break

            # CHECK FOR UNPATTERNED NON-CONVERGENCE #

            if iteration_count == 50:

                if var(sigma_list[25:]) >= var(sigma_list[:25]):

                    print("Sigma unpatterned Non-convergence")

                    new_sigma_v = mean(sigma_list[25:])

                else:

                    print("Sigma unpatterned divergence")

                    new_sigma_v = sigma_list[0] # With unpatterned divergence, we just take the original sigma estimate.

                non_covergence_switch = 1

                break

            # APPEND TO SIGMA LIST #

            sigma_list.append(eightSF_new_sigma_v)

            # NEW ESTIMATE OF DAILY DRIFT #

            log_vs = log(df_cd["v"]).values

            log_V_0 = log_vs[~isnan(log_vs)][0]

            log_V_N = log_vs[~isnan(log_vs)][-1]

            mu = (log_V_N - log_V_0)/nr_diff_obs

            # NEW ESTIMATE OF SIGMA_V #

            old_sigma_v = new_sigma_v # For checking convergence in next loop

            delta_log_vs = (log_vs - roll(log_vs, 1))[~isnan(log_vs - roll(log_vs, 1))]

            new_sigma_v = std(delta_log_vs - mu)

            iteration_count = iteration_count + 1

        ## CALCULATE DISTANCE TO, PROBABILITY OF DEFAULT ##

        debt = df_cd["debt"].values[-1] # Last value of debt

        if sigma_loop_switch == 0 and non_convergence_switch == 0 and estimation_failure_switch == 0 and zero_sigma_switch == 0:

            log_value = log(df_cd["v"][df_cd["v"].notnull()].values[-1]) # We carry forward the previous value of value if  
                                                                        # equity is missing for the final observation.

        elif estimation_failure_switch == 1 or zero_sigma_switch == 1:

            None # We can't really do anything if our estimation has failed, and don't need to do anything if default has
                # probability 0.

        else:

            equity = df_cd["equity"][df_cd["equity"].notnull()].values[-1] # Last valid value of equity

            r = df_cd["dgs1"].values[-1] # Last value of r

            v = None # Initiate value of v

            for coef in gammas:

                if fsolve(find_v, equity + coef*debt, full_output = True)[2] == 1:

                    v = fsolve(find_v, equity + coef*debt)[0]

                    break

            if v == None:

                try: # Last resort for starting guess employed. Probably not sustainable.

                    if fsolve(find_v, prev_v, full_output = True)[2] == 1:

                        print("Last resort employed")

                        v = fsolve(find_v, prev_v)[0]

                    else:

                        print("Estimation failure")

                        v = None

                        estimation_failure_switch = 1

                except:

                    print("Estimation failure")

                    v = None

                    estimation_failure_switch = 1

            if estimation_failure_switch == 0:

                log_value = log(v)

        if debt == 0 or zero_sigma_switch == 1: # If the face value of debt is 0, then distance to default is infinite 
                                               # and probability of default is zero. Same with zero variance in market value
            
            dd = nan # Distance to default

            p_d = 0 # Probability of default

        elif estimation_failure_switch == 1:

            dd = nan

            p_d = nan

        else:

            log_debt = log(debt)

            dd = (log_value - log_debt + (mu - 0.5*(new_sigma_v**2))*t)/(new_sigma_v*(t**0.5))

            p_d = norm.cdf(-1*dd)

        ## POPULATE NEW DATAFRAME ##

        df_dd.loc[counter] = [cd[0], cd[1], dd, p_d] # cusip; datadate; distance_to_default; probability_of_default


        ## UPDATE COUNTER ##

        counter = counter + 1


        ## PRINT SOME SUMMARY STATISTICS ##

        if counter % 20 == 0:

            time2 = time() - time1

            print(tranche_nr, counter, dd, p_d, time2)

            time1 = time()

            if counter % 200 == 0:

                df_dd.to_stata("001d_dd_complete_tranche" + str(tranche_nr) + ".dta", convert_dates = {"datadate":"td"})

        else: print(tranche_nr, counter, dd, p_d)


    ##########################
    # EXPORT FINAL DATAFRAME #
    ##########################

    df_dd.to_stata("001d_dd_complete_tranche" + str(tranche_nr) + ".dta", convert_dates = {"datadate":"td"})